In [1]:
pip install tensorflow==2.9


ERROR: Could not find a version that satisfies the requirement tensorflow==2.9 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.9


In [2]:
pip install tensorflow==2.17.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.17.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.17.0 which is incompatible.


In [3]:
pip install tensorflow


In [5]:
import tensorflow as tf
from tensorflow.experimental import dtensor

# Check the number of available logical devices
tf.config.list_logical_devices('CPU')  # For CPU setup


[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [6]:
from tensorflow.experimental import dtensor

# Define your devices (in this case, 8 virtual CPUs)
devices = [f'CPU:{i}' for i in range(8)]
mesh = dtensor.create_mesh([("batch", 8)], devices=devices)


In [7]:
class MLP(tf.Module):
    def __init__(self, dense_layouts):
        super().__init__()
        self.dense1 = Dense(1200, 48, (1, 2), dense_layouts[0], activation=tf.nn.relu)
        self.bn = BatchNorm()
        self.dense2 = Dense(48, 2, (3, 4), dense_layouts[1])

    def __call__(self, x):
        y = x
        y = self.dense1(y)
        y = self.bn(y)
        y = self.dense2(y)
        return y


In [8]:
def repack_batch(x, y, mesh):
    x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
    y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
    return x, y


In [9]:
@tf.function
def train_step(model, x, y, learning_rate=tf.constant(1e-4)):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    gradients = tape.gradient(loss, model.trainable_variables)
    for parameter, parameter_gradient in zip(model.trainable_variables, gradients):
        parameter.assign_sub(learning_rate * parameter_gradient)
    accuracy = 1.0 - tf.reduce_sum(tf.cast(tf.argmax(logits, axis=-1, output_type=tf.int64) != y, tf.float32)) / x.shape[0]
    return {'loss': loss, 'accuracy': accuracy}


In [11]:
import tempfile
import tensorflow as tf

CHECKPOINT_DIR = tempfile.mkdtemp()  # Temporary directory to store checkpoints

def start_checkpoint_manager(model):
    # Create a checkpoint for the model
    ckpt = tf.train.Checkpoint(root=model)

    # Create the checkpoint manager to manage checkpoint files
    manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_DIR, max_to_keep=3)

    # If there's an existing checkpoint, restore it
    if manager.latest_checkpoint:
        print("Restoring from checkpoint")
        ckpt.restore(manager.latest_checkpoint).assert_consumed()
    else:
        print("Starting new training")

    return manager


In [15]:
# Skip the virtual CPU configuration if it's not necessary for your use case
DEVICES = tf.config.list_physical_devices('CPU')  # Use available CPU devices
print(DEVICES)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [17]:
!pip install tensorflow==2.18.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0


In [1]:
!python --version



Python 3.11.11


In [2]:
import tensorflow as tf
from tensorflow.experimental import dtensor

# Define the function to configure virtual CPUs
def configure_virtual_cpus(ncpu):
    phy_devices = tf.config.list_physical_devices('CPU')
    tf.config.set_logical_device_configuration(phy_devices[0], [
          tf.config.LogicalDeviceConfiguration()  # Adjust this for the number of CPUs
      ] * ncpu)

# Call the function to configure 8 virtual CPUs
configure_virtual_cpus(8)

# Define the list of devices (logical devices in your system)
DEVICES = [f'CPU:{i}' for i in range(8)]

# Verify the logical devices
print(tf.config.list_logical_devices('CPU'))

# Create mesh for data parallel training using the logical devices
mesh = dtensor.create_mesh([("batch", 8)], devices=DEVICES)

# Model definition (MLP with Dense and BatchNorm layers)
class Dense(tf.Module):
    def __init__(self, input_size, output_size, init_seed, weight_layout, activation=None):
        super().__init__()

        random_normal_initializer = tf.function(tf.random.stateless_normal)

        self.weight = dtensor.DVariable(
            dtensor.call_with_layout(
                random_normal_initializer, weight_layout,
                shape=[input_size, output_size],
                seed=init_seed
            ))
        if activation is None:
            activation = lambda x: x
        self.activation = activation

        bias_layout = weight_layout.delete([0])

        self.bias = dtensor.DVariable(
            dtensor.call_with_layout(tf.zeros, bias_layout, [output_size]))

    def __call__(self, x):
        y = tf.matmul(x, self.weight) + self.bias
        y = self.activation(y)
        return y

# BatchNorm Layer
class BatchNorm(tf.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, x, training=True):
        mean, variance = tf.nn.moments(x, axes=[0])
        return tf.nn.batch_normalization(x, mean, variance, 0.0, 1.0, 1e-5)

# Multi-layer Perceptron (MLP)
class MLP(tf.Module):
    def __init__(self, dense_layouts):
        super().__init__()

        self.dense1 = Dense(1200, 48, (1, 2), dense_layouts[0], activation=tf.nn.relu)
        self.bn = BatchNorm()
        self.dense2 = Dense(48, 2, (3, 4), dense_layouts[1])

    def __call__(self, x):
        y = x
        y = self.dense1(y)
        y = self.bn(y)
        y = self.dense2(y)
        return y

# Instantiate the model with data parallel training mesh
model = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),
             dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)])

print("Model is defined.")


[LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:CPU:1', device_type='CPU'), LogicalDevice(name='/device:CPU:2', device_type='CPU'), LogicalDevice(name='/device:CPU:3', device_type='CPU'), LogicalDevice(name='/device:CPU:4', device_type='CPU'), LogicalDevice(name='/device:CPU:5', device_type='CPU'), LogicalDevice(name='/device:CPU:6', device_type='CPU'), LogicalDevice(name='/device:CPU:7', device_type='CPU')]


/usr/local/lib/python3.11/dist-packages/tensorflow/dtensor/python/layout.py:242: UserWarning: MessageFactory class is deprecated. Please use GetMessageClass() instead of MessageFactory.GetPrototype. MessageFactory class will be removed after 2024.
  return hash(self.as_proto().SerializeToString(deterministic=True))


Model is defined.
